# The NBA's Next Super Star

## Abstract
All NBA superstars have their rookie seasons. No matter it started with a [terrible debut year (eg. Lebron James)](http://www.thesportster.com/basketball/15-nba-superstars-who-were-terrible-in-their-debut-season/) or rewarding one like being selected to the [all-star game (eg. Tim Duncan)](http://www.nba-allstar.com/players/lists/all-star-game-rookies.htm), it is believed that there exists some evidences in their early years foretelling their future success. In this project, we will study and examine those key factors that drives the players towards a successful career, and eventually predicting the future super star in NBA.

## Problem Statement
A common way to foresee a successful NBA player from his rookie year is normally by looking at the draft order. However, such argument is incomprehensible for not taking other extrinsic factors into account.

## Introduction
This project aims to predict the career growth of an NBA player with the aid of the following data sources that reflects a player's performance, annual ratings on video games, and exposure on social media and newspaper.

1. Performance: The past and current on-court statistics on-court. (eg. [1991 NBA Draft](http://www.basketball-reference.com/draft/NBA_1991.html))

2. Ratings: The player ratings in the NBA 2K16 video game (eg. [2K16 MyTEAM Players](http://2kmtcentral.com/16/players/theme/dynamic))

3. Social Media: The amount of tweets of a player might reflect his attitude and seriousness of his career. Besides, a language model composed of the tweet content could be useful.(eg. [Twitter API](https://dev.twitter.com/rest/reference/get/statuses/user_timeline))

4. Related News: As news carries information such as fans' support, anticipation, expert analysis, injury report, and anticipation, the language model built accordingly would be a great indicator.

## Objective
Judging from indicators such as efficiency or other stats on-court, we would like to predict the likelihood of a player becoming a superstar that dominates the league in the future. 

## Import Libraries

In [ ]:
from urllib import urlopen
from bs4 import BeautifulSoup
import pandas as pd

## Data Preparation